**Import CSV Files**

In [27]:
import pandas as pd
ground_truth = pd.read_csv('val_annotations.csv')
pred = pd.read_csv('predict_val_frcnn.csv')

**Make dictionary from dataframe**

In [28]:
def dictmaker(df):
    
    data_dict = {}
    #initializing prev which is used to check if consecutive rows are from the same image
    prev = ''
    for ind in df.index:
        name = df['filename'][ind]
        if prev != name:
            # initialize a new bbox list if the current row represents a new image
            bboxes =[]
            
        prev = name

        xmin = df['xmin'][ind]
        ymin = df['ymin'][ind]
        xmax = df['xmax'][ind]
        ymax = df['ymax'][ind]

        bbox = [xmin, ymin, xmax, ymax]
        bboxes.append(bbox)
        
        #store in dictionary
        data_dict[name] = bboxes

    return data_dict


In [29]:
#creating dictionary from ground truth and prediction dataframes
gnd = dictmaker(ground_truth)
predict = dictmaker(pred)

In [30]:
#Gets list of unique file names in the set
names = ground_truth['filename'].unique()
names = list(names)

In [31]:
### IoU calculation code taken from Treesfive's git repo - https://github.com/Treesfive/calculate-iou

import numpy as np

iou_thres = 0.5

def get_iou(pred_box, gt_box):
    """
    pred_box : the coordinate for predict bounding box
    gt_box :   the coordinate for ground truth bounding box
    return :   the iou score
    the  left-down coordinate of  pred_box:(pred_box[0], pred_box[1])
    the  right-up coordinate of  pred_box:(pred_box[2], pred_box[3])
    """
    # 1.get the coordinate of inters
    ixmin = max(pred_box[0], gt_box[0])
    ixmax = min(pred_box[2], gt_box[2])
    iymin = max(pred_box[1], gt_box[1])
    iymax = min(pred_box[3], gt_box[3])

    iw = np.maximum(ixmax-ixmin+1., 0.)
    ih = np.maximum(iymax-iymin+1., 0.)

    # 2. calculate the area of inters
    inters = iw*ih

    # 3. calculate the area of union
    uni = ((pred_box[2]-pred_box[0]+1.) * (pred_box[3]-pred_box[1]+1.) +
           (gt_box[2] - gt_box[0] + 1.) * (gt_box[3] - gt_box[1] + 1.) -
           inters)

    # 4. calculate the overlaps between pred_box and gt_box
    iou = inters / uni

    return iou

**Calculate True Positive, False Positve and False Negative**

In [32]:
#initializing global variables
TP = 0
FP = 0
FN = 0

#iterate through each file
for name in names:
    #check if the file is in the prediciton list
    try:
        gnd_array = gnd[name]
        pred_array = predict[name]
        
        #initialize tp, fp, fn related variables for one image
        image_true_pred = 0
        image_total_pred = len(pred_array)
        image_fn = 0
        image_fp = 0
        image_tp = 0

        #iterate over all the ground truth bboxes
        for bbox_gnd in gnd_array:
            all_bbox_iou = []
            
            #calculate iou of each ground truth bouding box for all the prediction bboxes
            for bbox_pred in pred_array:
                all_bbox_iou.append(get_iou(bbox_pred, bbox_gnd))

            max_iou = max(all_bbox_iou)

            #check if the max iou for a ground truth bbox meets the threshold
            if max_iou >= iou_thres:
                image_true_pred += 1
            else:
                image_fn += 1

        #false positives are calculated after all true predicitons have been identified
        image_fp += (image_total_pred - image_true_pred)
        image_tp = image_true_pred

        #incrementing global values
        TP += image_tp
        FP += image_fp
        FN += image_fn

    #if a file is not in the prediciton list then all the ground truth bboxes are false negatives
    except:
        FN += len(gnd[name])
    


**Calculate Precision, Recall and F1-Score**

In [33]:
pre = TP/(TP+FP)
rec = TP/(TP+FN)
f1_score = (2*pre*rec)/(pre+rec)

pre, rec, f1_score

(0.768, 0.7868852459016393, 0.7773279352226721)